In [ ]:
# import sys
# import subprocess

# if "google.colab" in sys.modules:
#     ## 폰트 관련
#     subprocess.run(["sudo", "apt-get", "install", "-y", "fonts-nanum"])
#     subprocess.run(["sudo", "fc-cache", "-fv"])
#     subprocess.run(["rm", "-rf", "~/.cache/matplotlib"]) 
#     # KoBERT와 sentencepiece 설치
#     subprocess.run(["pip", "install", "git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf"])
#     subprocess.run(["pip", "install", "sentencepiece"])
#     subprocess.run(["pip", "install", "datasets"])

In [1]:
import sys
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    path = '/content/drive/Othercomputers/내 MacBook Air/Documents/personal_llm_projects/dacon_merong/'
else:
    path = '../'

data_path =  path + 'data/' + '{}.csv'

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
# 한글 폰트 문제 해결
# matplotlib은 한글 폰트를 지원하지 않음
# os정보
import platform

# font_manager : 폰트 관리 모듈
# rc : 폰트 변경 모듈
from matplotlib import font_manager, rc
# unicode 설정
plt.rcParams['axes.unicode_minus'] = False

if 'google.colab' in sys.modules:
    # 폰트 경로 찾기
    plt.rc('font', family='NanumBarunGothic')
    plt.rcParams['axes.unicode_minus'] =False

elif platform.system() == 'Darwin':
    rc('font', family='AppleGothic') # os가 macos
elif platform.system() == 'Windows':
    path = 'c:/Windows/Fonts/malgun.ttf' # os가 windows
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print("Unknown System")

### Reactive Optimizer for DeepLearning

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# 깊이 5의 MLP 모델 정의 (Dropout 포함)
class DeepANN(nn.Module):
    def __init__(self, input_size=28*28, hidden_size=128, output_size=1, dropout_prob=0.0):
        super(DeepANN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size // 2)
        self.fc3 = nn.Linear(hidden_size // 2, hidden_size // 4)
        # self.fc4 = nn.Linear(hidden_size // 4, hidden_size // 8)
        self.fc5 = nn.Linear(hidden_size // 2, output_size)
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(F.relu(self.fc2(x)))
        # x = self.dropout(F.relu(self.fc3(x)))  # Dropout은 여기까지만 적용
        # x = F.relu(self.fc4(x))
        x = self.fc5(x)  # 마지막 레이어에는 활성화 함수 X
        return x

# 하이퍼파라미터 설정
input_size = 10
hidden_size = 128
output_size = 1
learning_rate = 0.001  # 학습률 감소

# 모델, 손실 함수, 옵티마이저 초기화
model = DeepANN(input_size, hidden_size, output_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 더미 데이터 생성
x_train = torch.ones(100, input_size) * (torch.rand([100,input_size]) * 20)
x_mean = x_train.mean(dim=1)
# y_train = ((x_train.mean(dim=1) * 3 + 5) + torch.randn([x_train.shape[0]]) *0.1  ).unsqueeze(1)  # 차원 맞추기
y_train = ((3* x_mean * x_mean + 5 * x_mean +1)+ torch.randn([x_train.shape[0]]) *0.1  ).unsqueeze(1)  # 차원 맞추기

# 학습 과정
epochs = 5000
for epoch in range(epochs):
    model.train()  # 학습 모드
    optimizer.zero_grad()
    outputs = model(x_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 500 == 0:  # 500번마다 출력
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

print("학습 완료!")



Epoch [500/5000], Loss: 2614.1794
Epoch [1000/5000], Loss: 1266.7375
Epoch [1500/5000], Loss: 633.9652
Epoch [2000/5000], Loss: 402.0826
Epoch [2500/5000], Loss: 232.8163
Epoch [3000/5000], Loss: 114.7139
Epoch [3500/5000], Loss: 44.6261
Epoch [4000/5000], Loss: 7.0094
Epoch [4500/5000], Loss: 1.4004
Epoch [5000/5000], Loss: 0.3564
학습 완료!


In [14]:
# 예측 (평가 모드)
model.eval()
test_input  = torch.ones([1, input_size]) * 3
test_mean = test_input.mean(dim=1)
with torch.no_grad():
    pred = model(test_input)

print('예측값:', pred.item())
print('실제값:', (3 * test_mean * test_mean + 5 * test_mean +1).item())


예측값: 2.098465919494629
실제값: 43.0


In [ ]:
temp = torch.tensor([1,2]).unsqueeze(0)

temp

tensor([[1, 2]])

In [166]:
wow =  next(iter(model.fc1.parameters()))

In [157]:
wow

Parameter containing:
tensor([[ 0.0267,  0.1749,  0.3378,  ...,  0.3572, -0.0099,  0.3429],
        [-0.0911, -0.1270, -0.0851,  ..., -0.2633,  0.4055,  0.1302],
        [ 0.3075,  0.3172,  0.1539,  ..., -0.1694, -0.2756, -0.1516],
        ...,
        [ 0.3017, -0.2872,  0.1753,  ...,  0.1161, -0.5035, -0.1019],
        [ 0.0150,  0.1756,  0.0742,  ...,  0.1252,  0.2372,  0.0529],
        [-0.1688,  0.1289, -0.2143,  ..., -0.0678,  0.0393,  0.3430]],
       requires_grad=True)